<a href="https://colab.research.google.com/github/Benyormin/Question_answering/blob/main/Copy_of_AI_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependencies

In [ ]:
!pip install "datasets<4.0.0"
!pip install -U "transformers>=4.35.0" bitsandbytes accelerate peft torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 36.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.10.0
    Uninstalling accelerate-1.10.0:
      Successfully uninstalled accelerate-1.10.0
  Attempt

In [ ]:
from datasets import load_dataset

ds = load_dataset("SajjadAyoubi/persian_qa")

README.md: 0.00B [00:00, ?B/s]

persian_qa.py: 0.00B [00:00, ?B/s]

persian_qa/train/0000.parquet:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/165k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/930 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})

In [ ]:
ds['train'][0]

{'id': 1,
 'title': 'شرکت فولاد مبارکه اصفهان',
 'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده\u200cاست. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده\u200cرود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم\u200cآبی زاینده\u200cرود شناخته می\u200cشود.',
 'questio

In [ ]:
splited = ds["train"].train_test_split(test_size=0.1, seed= 42)
train_ds = splited['train']
val_ds = splited['test']
test_ds = ds["validation"]

In [ ]:
train_ds = train_ds.shuffle(seed=42).select(range(4000))
train_ds

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 4000
})

In [ ]:
val_ds

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 901
})

In [ ]:
test_ds

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 930
})

### Tokenizer

In [ ]:
from transformers import AutoTokenizer

model_id = "unsloth/llama-3-8b-bnb-4bit"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# Llama 3.x has no dedicated PAD; use EOS for padding during training
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.truncation_side = "right"  # truncate tail of the prompt first if too long


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [ ]:
from typing import Any, Dict, List
import math

def _extract_gold_and_start(ans: Any):
    """
    Robustly extract answer text and char start from a variety of shapes:
    - ans might be a dict like {'text': [...], 'answer_start':[...]}
    - ans might be a list of dicts [{'text':[...], 'answer_start':[...]}, ...]
    - or weird/missing cases.
    Returns: (gold_text: str or None, answer_start: int or None)
    """
    # default fallback
    NO_ANS_STR = "پاسخی در متن موجود نیست"

    # If it's a dict (typical HF QA format)
    if isinstance(ans, dict):
        texts = ans.get("text") or []   # could be []
        starts = ans.get("answer_start") or []
        if isinstance(texts, list) and len(texts) > 0 and texts[0]:
            gold = texts[0]
            start = starts[0] if isinstance(starts, list) and len(starts) > 0 else None
            return gold, start
        else:
            return None, None

    # If it's a list (sometimes answers stored as list of dicts)
    if isinstance(ans, list) and len(ans) > 0:
        first = ans[0]
        if isinstance(first, dict):
            texts = first.get("text") or []
            starts = first.get("answer_start") or []
            if isinstance(texts, list) and len(texts) > 0 and texts[0]:
                return texts[0], (starts[0] if isinstance(starts, list) and len(starts) > 0 else None)
        # maybe a list of strings
        if isinstance(first, str) and first.strip():
            return first, None

    return None, None


def preprocess_batch(
    examples: Dict[str, List],
    max_length: int = 512,
    max_context_chars: int = 2000,
    answer_window: int = 400,
    drop_missing_answers: bool = False,   # set True to filter examples with no gold
):
    """
    Batched map function returning input_ids, labels, attention_mask lists.
    Safe against empty/missing answers.
    """
    prompts = []
    targets = []

    questions = examples["question"]
    contexts = examples["context"]
    answers = examples["answers"]

    # keep an index map when we want to drop missing answers
    keep_mask = []

    for q, c, ans in zip(questions, contexts, answers):
        gold, start = _extract_gold_and_start(ans)

        if gold is None:
            # No gold available
            if drop_missing_answers:
                keep_mask.append(False)
                continue
            else:
                gold = "پاسخی در متن موجود نیست"
                start = None
                keep_mask.append(True)
        else:
            keep_mask.append(True)

        # slice context around answer_start if available
        if start is not None and isinstance(start, int) and 0 <= start < len(c):
            left = max(0, start - answer_window)
            right = min(len(c), start + len(gold) + answer_window)
            c_window = c[left:right]
        else:
            c_window = c if len(c) <= max_context_chars else c[:max_context_chars]

        # Build a simple Persian instruction prompt
        prompt = (
            "دستور: فقط بر اساس متن موجود، پاسخ کوتاه بده.\n\n"
            f"متن:\n{c_window}\n\n"
            f"سوال: {q}\n\n"
            "پاسخ:"
        )

        target = gold  # we'll append eos later when tokenizing if desired

        prompts.append(prompt)
        targets.append(target)

    # If we dropped examples, the returned lists simply skip them. Datasets.map expects same-length outputs
    # as the inputs when not removing columns. But we use remove_columns so this is OK.
    # Tokenize
    prompt_tok = tokenizer(
        prompts,
        add_special_tokens=False,
        truncation=True,
        max_length=max_length,
        return_attention_mask=True
    )
    target_tok = tokenizer(
        [t + tokenizer.eos_token for t in targets],   # add eos to targets
        add_special_tokens=False,
        truncation=True,
        max_length=128
    )

    input_ids_b = []
    labels_b = []
    attention_mask_b = []

    for p_ids, a_ids, p_attn in zip(prompt_tok["input_ids"], target_tok["input_ids"], prompt_tok["attention_mask"]):
        # ensure combined length <= max_length, keep answer intact if possible
        if len(p_ids) + len(a_ids) > max_length:
            keep_prompt = max_length - len(a_ids)
            if keep_prompt <= 0:
                # answer too long for max_length -> truncate answer (unlikely for short answers)
                a_ids = a_ids[:max_length]
                p_ids = []
            else:
                p_ids = p_ids[:keep_prompt]

        ids = p_ids + a_ids
        labels = [-100] * len(p_ids) + a_ids
        attn = [1] * len(ids)

        input_ids_b.append(ids)
        labels_b.append(labels)
        attention_mask_b.append(attn)

    return {
        "input_ids": input_ids_b,
        "labels": labels_b,
        "attention_mask": attention_mask_b,
    }


In [ ]:
cols_to_remove = list(train_ds.column_names)  # we’ll replace with model-ready tensors

train_tok = train_ds.map(
    preprocess_batch,
    batched=True,
    remove_columns=cols_to_remove,
    desc="Tokenizing train",
)

cols_to_remove = list(val_ds.column_names)

val_tok = val_ds.map(
    preprocess_batch,
    batched=True,
    remove_columns=cols_to_remove,
    desc="Tokenizing val",
)

cols_to_remove = list(test_ds.column_names)

test_tok = test_ds.map(
    preprocess_batch,
    batched=True,
    remove_columns=cols_to_remove,
    desc="Tokenizing test",
)


Tokenizing train:   0%|          | 0/8107 [00:00<?, ? examples/s]

Tokenizing val:   0%|          | 0/901 [00:00<?, ? examples/s]

Tokenizing test:   0%|          | 0/930 [00:00<?, ? examples/s]

In [ ]:
train_ds

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 8107
})

In [ ]:
train_tok[0]

{'input_ids': [13628,
  47686,
  59756,
  25,
  105577,
  100317,
  110975,
  109123,
  106726,
  69885,
  110703,
  123750,
  118512,
  382,
  10386,
  102017,
  512,
  12061,
  53804,
  68483,
  32482,
  83711,
  102529,
  118731,
  113815,
  46677,
  100656,
  32482,
  45430,
  106698,
  33411,
  102825,
  100885,
  51343,
  68483,
  32482,
  101646,
  102270,
  77941,
  110267,
  5821,
  105082,
  77941,
  118648,
  111116,
  12942,
  101516,
  38624,
  88517,
  102172,
  101599,
  100490,
  46693,
  48732,
  100467,
  105463,
  94253,
  13,
  109282,
  51343,
  68483,
  32482,
  113538,
  220,
  119675,
  113870,
  106987,
  38624,
  106461,
  118731,
  100490,
  51343,
  68483,
  101613,
  94253,
  13,
  108996,
  120136,
  100408,
  103184,
  100498,
  68483,
  101269,
  51343,
  68483,
  32482,
  94253,
  38624,
  109271,
  83711,
  120725,
  38624,
  111116,
  100550,
  20665,
  106260,
  94253,
  13,
  105636,
  112911,
  16552,
  102261,
  51343,
  68483,
  32482,
  100317,


In [ ]:
import gc, torch, os
gc.collect()
torch.cuda.empty_cache()
# Optionally try the allocator tweak (see notes below)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "garbage_collection_threshold:0.6,max_split_size_mb:128"
# or try the suggestion from the error (experimental):
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
  Attempting uninstall: datasets
    Found existing installation: datasets 3.6.0
    Uninstalling datasets-3.6.0:
      Successfully uninstalled datasets-3.6.0
  Attempting uninstall: peft
    Found existing installation: peft 0.17.0
    Uninstalling peft-0.17.0:
      Successfully uninstalled peft-0.17.0


### Configure

In [ ]:
import math
import torch
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from tqdm.auto import tqdm

# ---- Config ----
MODEL_ID = "unsloth/llama-3-8b-bnb-4bit"
SAVE_DIR = "./lora_unsloth_4bit"
BATCH_SIZE = 1               # per step batch (before grad accumulation) — tune for your GPU
GRAD_ACCUM_STEPS = 16          # effective batch = BATCH_SIZE * GRAD_ACCUM_STEPS
NUM_EPOCHS = 1
LR = 2e-4
WEIGHT_DECAY = 0.01
WARMUP_STEPS = 50
MAX_GRAD_NORM = 1.0
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# ---- Load quantized model ----
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,   # use float16 compute
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",              # nf4 is common; try "fp4" if available
)
max_mem = {0: "13GiB", "cpu": "60GiB"}

print("Loading quantized model (this can take a minute)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",        # let HF auto-place layers across GPUs/CPU
    trust_remote_code=True,   # required for some community checkpoints
     max_memory=max_mem,
    low_cpu_mem_usage=True,
)

# ---- Prepare for k-bit training and add LoRA ----
# This will:
#  - make some modules trainable (enable_input_require_grads),
#  - set gradient_checkpointing if desired, etc.
model = prepare_model_for_kbit_training(model)

# LoRA config - adjust target_modules depending on model internals (Llama-like: q_proj/v_proj)
lora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # consider ["q_proj","k_proj","v_proj","o_proj"] if needed
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("PEFT/LoRA parameters:")
for n, p in model.named_parameters():
    print(n, p.numel(), p.requires_grad)



Loading quantized model (this can take a minute)...


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

PEFT/LoRA parameters:
base_model.model.model.embed_tokens.weight 525336576 False
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight 8388608 False
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight 16384 True
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight 16384 True
base_model.model.model.layers.0.self_attn.k_proj.weight 2097152 False
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight 2097152 False
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight 16384 True
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight 4096 True
base_model.model.model.layers.0.self_attn.o_proj.weight 8388608 False
base_model.model.model.layers.0.mlp.gate_proj.weight 29360128 False
base_model.model.model.layers.0.mlp.up_proj.weight 29360128 False
base_model.model.model.layers.0.mlp.down_proj.weight 29360128 False
base_model.model.model.layers.0.input_layernorm.weight 4096 False
base_model.model.model

In [ ]:
BATCH_SIZE = 1
# ---- Build DataLoaders ----
# Assumes train_tok / val_tok are datasets with columns: input_ids, attention_mask, labels (lists)
def collate_fn(batch):
    # batch is list of dicts with lists of ints
    input_ids = [b["input_ids"] for b in batch]
    attention_mask = [b["attention_mask"] for b in batch]
    labels = [b["labels"] for b in batch]

    # tokenizer.pad will pad input_ids & attention_mask
    padded = tokenizer.pad(
        {"input_ids": input_ids, "attention_mask": attention_mask},
        padding=True,
        return_tensors="pt",
    )

    max_len = padded["input_ids"].shape[1]
    # pad labels with -100
    padded_labels = [l + [-100] * (max_len - len(l)) for l in labels]
    padded["labels"] = torch.tensor(padded_labels, dtype=torch.long)

    return {
        "input_ids": padded["input_ids"],
        "attention_mask": padded["attention_mask"],
        "labels": padded["labels"]
    }

train_loader = DataLoader(train_tok, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_tok, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = Dataloader(test_tok, batch_size= BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

NameError: name 'Dataloader' is not defined

In [ ]:
train_loader

### Training

In [ ]:

# ---- Optimizer & scheduler (only LoRA params will be trainable) ----
# Peft/LoRA ensures only adapter params are .requires_grad True
optim_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(optim_params, lr=LR, weight_decay=WEIGHT_DECAY)

total_steps = (len(train_loader) // GRAD_ACCUM_STEPS) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=total_steps)

# ---- Mixed precision scaler ----
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

# ---- Training loop ----
global_step = 0
model.train()

# Note: do NOT call model.to(device) when you used device_map="auto" — HF already placed layers.
# However the input tensors should be on the CUDA device (usually cuda:0).
# We'll move tensors to DEVICE (cuda if available).
for epoch in range(NUM_EPOCHS):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    running_loss = 0.0
    optimizer.zero_grad()

    for step, batch in enumerate(pbar):
        # move batch to device (usually cuda)
        batch = {k: v.to(DEVICE) for k, v in batch.items()}

        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available(), dtype=torch.float16):
            outputs = model(**batch)
            loss = outputs.loss
            # normalize for gradient accumulation
            loss = loss / GRAD_ACCUM_STEPS

        scaler.scale(loss).backward()

        if (step + 1) % GRAD_ACCUM_STEPS == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(optim_params, MAX_GRAD_NORM)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
            global_step += 1

        running_loss += loss.item() * GRAD_ACCUM_STEPS
        if global_step % 10 == 0:
            pbar.set_postfix({"loss": f"{running_loss / (global_step + 1):.4f}"})

    # ---- End epoch: run evaluation on validation ----
    model.eval()
    total_val_loss = 0.0
    nb_eval_steps = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available(), dtype=torch.float16):
                outputs = model(**batch)
                val_loss = outputs.loss
            total_val_loss += val_loss.item()
            nb_eval_steps += 1

    avg_val_loss = total_val_loss / max(1, nb_eval_steps)
    try:
        perplexity = math.exp(avg_val_loss)
    except OverflowError:
        perplexity = float("inf")

    print(f"Epoch {epoch+1} finished. Val loss: {avg_val_loss:.4f} | Perplexity: {perplexity:.2f}")

    model.train()

# ---- Save LoRA adapter weights (small) ----
# This saves only the PEFT adapter weights (recommended)
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("Saved LoRA adapter and tokenizer to", SAVE_DIR)


/tmp/ipython-input-504756956.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


Epoch 1/1:   0%|          | 0/4000 [00:00<?, ?it/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipython-input-504756956.py:28: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available(), dtype=torch.float16):
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the tw

Validation:   0%|          | 0/901 [00:00<?, ?it/s]

/tmp/ipython-input-504756956.py:56: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available(), dtype=torch.float16):


Epoch 1 finished. Val loss: 0.1878 | Perplexity: 1.21
Saved LoRA adapter and tokenizer to ./lora_unsloth_4bit


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# zip and copy to Drive (Colab)
!zip -r lora_unsloth_4bit.zip lora_unsloth_4bit
!cp lora_unsloth_4bit.zip "/content/drive/MyDrive/Colab Notebooks/AI_task"

updating: lora_unsloth_4bit/ (stored 0%)
updating: lora_unsloth_4bit/tokenizer.json (deflated 85%)
updating: lora_unsloth_4bit/adapter_model.safetensors (deflated 7%)
updating: lora_unsloth_4bit/README.md (deflated 65%)
updating: lora_unsloth_4bit/special_tokens_map.json (deflated 64%)
updating: lora_unsloth_4bit/adapter_config.json (deflated 55%)
updating: lora_unsloth_4bit/tokenizer_config.json (deflated 96%)


In [ ]:
test_tok[0]

{'input_ids': [13628,
  47686,
  59756,
  25,
  105577,
  100317,
  110975,
  109123,
  106726,
  69885,
  110703,
  123750,
  118512,
  382,
  10386,
  102017,
  512,
  104456,
  100800,
  107856,
  112420,
  32482,
  117850,
  66143,
  100727,
  106830,
  116621,
  103418,
  107856,
  94253,
  100360,
  77941,
  117850,
  66143,
  69885,
  88517,
  102172,
  101599,
  114385,
  119396,
  102940,
  102218,
  13,
  112420,
  32482,
  115682,
  116241,
  109101,
  104793,
  107856,
  114385,
  119396,
  38624,
  115682,
  116241,
  109101,
  104793,
  107856,
  124891,
  38624,
  115682,
  116241,
  109101,
  107856,
  60942,
  92435,
  102261,
  220,
  101379,
  123738,
  82868,
  105708,
  102714,
  21604,
  5821,
  94253,
  13,
  100490,
  101440,
  108769,
  54810,
  33411,
  101141,
  220,
  100882,
  101009,
  104151,
  121147,
  108089,
  77941,
  57894,
  101613,
  64832,
  5821,
  69885,
  220,
  101950,
  121147,
  108089,
  77941,
  106622,
  68483,
  5821,
  105905,
  106637

## Load Model and test

In [ ]:
!pip install "datasets<4.0.0"
!pip install -U "transformers>=4.35.0" bitsandbytes accelerate peft torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 37.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.10.0
    Uninstalling accelerate-1.10.0:
      Successfully uninstalled accelerate-1.10.0
  Attempt

In [ ]:
from datasets import load_dataset

ds = load_dataset("SajjadAyoubi/persian_qa")

README.md: 0.00B [00:00, ?B/s]

persian_qa.py: 0.00B [00:00, ?B/s]

persian_qa/train/0000.parquet:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/165k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/930 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})

In [ ]:
ds['train'][0]

{'id': 1,
 'title': 'شرکت فولاد مبارکه اصفهان',
 'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده\u200cاست. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده\u200cرود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم\u200cآبی زاینده\u200cرود شناخته می\u200cشود.',
 'questio

In [ ]:
splited = ds["train"].train_test_split(test_size=0.1, seed= 42)
train_ds = splited['train']
val_ds = splited['test']
test_ds = ds["validation"]

In [ ]:
train_ds = train_ds.shuffle(seed=42).select(range(4000))
train_ds

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 4000
})

In [ ]:
val_ds

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 901
})

In [ ]:
test_ds

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 930
})

In [ ]:
from transformers import AutoTokenizer

model_id = "unsloth/llama-3-8b-bnb-4bit"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# Llama 3.x has no dedicated PAD; use EOS for padding during training
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
tokenizer.truncation_side = "right"  # truncate tail of the prompt first if too long


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [ ]:
from typing import Any, Dict, List
import math

def _extract_gold_and_start(ans: Any):
    """
    Robustly extract answer text and char start from a variety of shapes:
    - ans might be a dict like {'text': [...], 'answer_start':[...]}
    - ans might be a list of dicts [{'text':[...], 'answer_start':[...]}, ...]
    - or weird/missing cases.
    Returns: (gold_text: str or None, answer_start: int or None)
    """
    # default fallback
    NO_ANS_STR = "پاسخی در متن موجود نیست"

    # If it's a dict (typical HF QA format)
    if isinstance(ans, dict):
        texts = ans.get("text") or []   # could be []
        starts = ans.get("answer_start") or []
        if isinstance(texts, list) and len(texts) > 0 and texts[0]:
            gold = texts[0]
            start = starts[0] if isinstance(starts, list) and len(starts) > 0 else None
            return gold, start
        else:
            return None, None

    # If it's a list (sometimes answers stored as list of dicts)
    if isinstance(ans, list) and len(ans) > 0:
        first = ans[0]
        if isinstance(first, dict):
            texts = first.get("text") or []
            starts = first.get("answer_start") or []
            if isinstance(texts, list) and len(texts) > 0 and texts[0]:
                return texts[0], (starts[0] if isinstance(starts, list) and len(starts) > 0 else None)
        # maybe a list of strings
        if isinstance(first, str) and first.strip():
            return first, None

    return None, None


def preprocess_batch(
    examples: Dict[str, List],
    max_length: int = 512,
    max_context_chars: int = 2000,
    answer_window: int = 400,
    drop_missing_answers: bool = False,   # set True to filter examples with no gold
):
    """
    Batched map function returning input_ids, labels, attention_mask lists.
    Safe against empty/missing answers.
    """
    prompts = []
    targets = []

    questions = examples["question"]
    contexts = examples["context"]
    answers = examples["answers"]

    # keep an index map when we want to drop missing answers
    keep_mask = []

    for q, c, ans in zip(questions, contexts, answers):
        gold, start = _extract_gold_and_start(ans)

        if gold is None:
            # No gold available
            if drop_missing_answers:
                keep_mask.append(False)
                continue
            else:
                gold = "پاسخی در متن موجود نیست"
                start = None
                keep_mask.append(True)
        else:
            keep_mask.append(True)

        # slice context around answer_start if available
        if start is not None and isinstance(start, int) and 0 <= start < len(c):
            left = max(0, start - answer_window)
            right = min(len(c), start + len(gold) + answer_window)
            c_window = c[left:right]
        else:
            c_window = c if len(c) <= max_context_chars else c[:max_context_chars]

        # Build a simple Persian instruction prompt
        prompt = (
            "دستور: فقط بر اساس متن موجود، پاسخ کوتاه بده.\n\n"
            f"متن:\n{c_window}\n\n"
            f"سوال: {q}\n\n"
            "پاسخ:"
        )

        target = gold  # we'll append eos later when tokenizing if desired

        prompts.append(prompt)
        targets.append(target)

    # If we dropped examples, the returned lists simply skip them. Datasets.map expects same-length outputs
    # as the inputs when not removing columns. But we use remove_columns so this is OK.
    # Tokenize
    prompt_tok = tokenizer(
        prompts,
        add_special_tokens=False,
        truncation=True,
        max_length=max_length,
        return_attention_mask=True
    )
    target_tok = tokenizer(
        [t + tokenizer.eos_token for t in targets],   # add eos to targets
        add_special_tokens=False,
        truncation=True,
        max_length=128
    )

    input_ids_b = []
    labels_b = []
    attention_mask_b = []

    for p_ids, a_ids, p_attn in zip(prompt_tok["input_ids"], target_tok["input_ids"], prompt_tok["attention_mask"]):
        # ensure combined length <= max_length, keep answer intact if possible
        if len(p_ids) + len(a_ids) > max_length:
            keep_prompt = max_length - len(a_ids)
            if keep_prompt <= 0:
                # answer too long for max_length -> truncate answer (unlikely for short answers)
                a_ids = a_ids[:max_length]
                p_ids = []
            else:
                p_ids = p_ids[:keep_prompt]

        ids = p_ids + a_ids
        labels = [-100] * len(p_ids) + a_ids
        attn = [1] * len(ids)

        input_ids_b.append(ids)
        labels_b.append(labels)
        attention_mask_b.append(attn)

    return {
        "input_ids": input_ids_b,
        "labels": labels_b,
        "attention_mask": attention_mask_b,
    }


In [ ]:

#!pip install -q bitsandbytes accelerate transformers peft safetensors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 17.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

ZIP_PATH = "/content/drive/MyDrive/Colab Notebooks/AI_task/lora_unsloth_4bit.zip"
UNZIP_DIR = "/content/lora_unsloth_4bit"

Mounted at /content/drive


In [ ]:
# create target dir and unzip
os.makedirs(UNZIP_DIR, exist_ok=True)
!unzip -q "{ZIP_PATH}" -d "{UNZIP_DIR}"
print("Files in", UNZIP_DIR, ":", os.listdir(UNZIP_DIR))

Files in /content/lora_unsloth_4bit : ['lora_unsloth_4bit']


In [ ]:
## load tokenizer:

from transformers import PreTrainedTokenizerFast
import json
import os

tok_json = "/content/lora_unsloth_4bit/lora_unsloth_4bit/tokenizer.json"
tok_cfg = "/content/lora_unsloth_4bit/lora_unsloth_4bit/tokenizer_config.json"

# 1) Load tokenizer from the JSON file (fast & robust)
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tok_json)

# 2) Apply settings from tokenizer_config.json if present
if os.path.exists(tok_cfg):
    cfg = json.load(open(tok_cfg, "r", encoding="utf-8"))
    # Common keys: unk_token, pad_token, bos_token, eos_token, ...
    for key in ("unk_token", "pad_token", "bos_token", "eos_token", "sep_token", "cls_token"):
        if key in cfg:
            setattr(tokenizer, key, cfg[key])

# Optional: inspect
print("Loaded tokenizer. vocab_size:", tokenizer.vocab_size)
print("Special tokens:", tokenizer.special_tokens_map)


Loaded tokenizer. vocab_size: 128000
Special tokens: {'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|end_of_text|>'}


In [ ]:
# move all files from nested dir up one level, then remove nested folder
!sh -c 'mv /content/lora_unsloth_4bit/lora_unsloth_4bit/* /content/lora_unsloth_4bit/ || true'
!rmdir /content/lora_unsloth_4bit/lora_unsloth_4bit || true

# verify
!ls -la /content/lora_unsloth_4bit


total 23560
drwxr-xr-x 2 root root     4096 Aug 26 19:49 .
drwxr-xr-x 1 root root     4096 Aug 26 19:49 ..
-rw-r--r-- 1 root root      861 Aug 25 11:14 adapter_config.json
-rw-r--r-- 1 root root  6832600 Aug 25 11:14 adapter_model.safetensors
-rw-r--r-- 1 root root     5208 Aug 25 11:14 README.md
-rw-r--r-- 1 root root      335 Aug 25 11:14 special_tokens_map.json
-rw-r--r-- 1 root root    50657 Aug 25 11:14 tokenizer_config.json
-rw-r--r-- 1 root root 17210059 Aug 25 11:14 tokenizer.json


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

MODEL_ID = "unsloth/llama-3-8b-bnb-4bit"   # must match original
PEFT_DIR = UNZIP_DIR  # path where adapter files were extracted

# Bits & Bytes config must match what you used when training
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,   # same as training
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

print("Loading base model (quantized) — this may take a minute...")
max_mem = {0: "13GiB", "cpu": "60GiB"}

base = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",         # HF will auto-place layers across GPUs/CPU
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    max_memory=max_mem
)



# apply adapter weights (PEFT)
print("Applying PEFT/LoRA adapter from", PEFT_DIR)
model = PeftModel.from_pretrained(base, PEFT_DIR, device_map="auto")
model.eval()


Loading base model (quantized) — this may take a minute...


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:231: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

Applying PEFT/LoRA adapter from /content/lora_unsloth_4bit


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

In [ ]:
prompt = "سوال: موقعیت جغرافیایی باشگاه فوتبال آرسنال را بگویید؟\nپاسخ:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

GEN_ARGS = dict(max_new_tokens=64, do_sample=False, num_beams=4, early_stopping=True)
with torch.no_grad():
    out = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs.get("attention_mask"), **GEN_ARGS)

generated = tokenizer.decode(out[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
print("=== GENERATED ===\n", generated)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== GENERATED ===
 سوال: موقعیت جغرافیایی باشگاه فوتبال آرسنال را بگویید؟
پاسخ:پاسخی در متن موجود نیست


In [ ]:
out

tensor([[128000,  20665, 105815,     25, 126428, 116480,  14728, 100982, 106830,
         107856, 100281, 102361,  12061,  32482, 100394, 106855, 100633,  66143,
         107315,  68483,  92444,  36344,     25,  68483,  92444, 112927,  77941,
         109123, 106726, 105182, 128001]], device='cuda:0')

In [ ]:
model(input_ids=inputs["input_ids"], attention_mask=inputs.get("attention_mask"))

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 4.9609,  6.5977, 10.0625,  ..., -3.3125, -3.3125, -3.3125],
         [ 1.0596, -0.1360,  1.2178,  ...,  0.1113,  0.1113,  0.1113],
         [ 3.9004,  3.7637,  6.7109,  ..., -4.1758, -4.1758, -4.1797],
         ...,
         [-4.9492, -4.1055, -0.6562,  ...,  0.9766,  0.9766,  0.9766],
         [ 4.1211,  5.3320,  5.8281,  ..., -1.7061, -1.7061, -1.7061],
         [ 0.7363, -0.0610,  1.6055,  ..., -1.6582, -1.6582, -1.6582]]],
       device='cuda:0', dtype=torch.float16), past_key_values=DynamicCache(layers=[<transformers.cache_utils.DynamicLayer object at 0x78633446ff20>, <transformers.cache_utils.DynamicLayer object at 0x786334441790>, <transformers.cache_utils.DynamicLayer object at 0x786334440620>, <transformers.cache_utils.DynamicLayer object at 0x786334443d70>, <transformers.cache_utils.DynamicLayer object at 0x78633446d550>, <transformers.cache_utils.DynamicLayer object at 0x786334440dd0>, <transformers.cache_utils.DynamicLayer

#########################################

In [ ]:
cols_to_remove = list(test_ds.column_names)
test_tok = test_ds.map(
    preprocess_batch,
    batched=True,
    remove_columns=cols_to_remove,
    desc="Tokenizing test",
)


Tokenizing test:   0%|          | 0/930 [00:00<?, ? examples/s]

In [ ]:
def collate_fn(batch):
    # batch is list of dicts with lists of ints
    input_ids = [b["input_ids"] for b in batch]
    attention_mask = [b["attention_mask"] for b in batch]
    labels = [b["labels"] for b in batch]

    # tokenizer.pad will pad input_ids & attention_mask
    padded = tokenizer.pad(
        {"input_ids": input_ids, "attention_mask": attention_mask},
        padding=True,
        return_tensors="pt",
    )

    max_len = padded["input_ids"].shape[1]
    # pad labels with -100
    padded_labels = [l + [-100] * (max_len - len(l)) for l in labels]
    padded["labels"] = torch.tensor(padded_labels, dtype=torch.long)

    return {
        "input_ids": padded["input_ids"],
        "attention_mask": padded["attention_mask"],
        "labels": padded["labels"]
    }

In [ ]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch
import re
import string
import collections
import csv

# ---------- Config ----------
TEST_BATCH_SIZE = 4
GEN_ARGS = dict(max_new_tokens=32, num_beams=1, do_sample=False, early_stopping=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# ----------------------------

# If you already have test_tok as a list/dataset of tokenized torch tensors:
test_loader = DataLoader(test_tok, batch_size=TEST_BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# ---------- helper functions for metrics ----------
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    s = s.lower()
    # remove punctuation
    s = "".join(ch for ch in s if ch not in set(string.punctuation))
    # remove articles
    s = re.sub(r'\b(a|an|the)\b', ' ', s)
    # normalize whitespace
    s = " ".join(s.split())
    return s

def exact_match_score(pred, truth):
    return normalize_answer(pred) == normalize_answer(truth)

def f1_score(pred, truth):
    pred_tokens = normalize_answer(pred).split()
    truth_tokens = normalize_answer(truth).split()
    if len(pred_tokens) == 0 and len(truth_tokens) == 0:
        return 1.0
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return 0.0
    common = collections.Counter(pred_tokens) & collections.Counter(truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(truth_tokens)
    return 2 * precision * recall / (precision + recall)

# flexible function to extract references from a batch element
def extract_references_from_item(item):
    """
    Try several common keys. Return a list of reference strings.
    """
    # If labels exist as token ids, try decode
    if "labels" in item:
        try:
            labs = item["labels"]
            if isinstance(labs, torch.Tensor):
                txt = tokenizer.decode(labs, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                return [txt]
        except Exception:
            pass

    # Common HF QA dataset structure: item["answers"] = {"text": [...], "answer_start": [...]}
    if "answers" in item:
        ans = item["answers"]
        if isinstance(ans, dict) and "text" in ans:
            texts = ans["text"]
            if isinstance(texts, list):
                return texts
            elif isinstance(texts, str):
                return [texts]

    # Common single-field names
    for k in ("answer", "answer_text", "gold_answer", "label_text", "target_text"):
        if k in item:
            v = item[k]
            if isinstance(v, list):
                return v
            elif isinstance(v, str):
                return [v]

    # try 'metadata' or 'extras' fields
    if "meta" in item and isinstance(item["meta"], dict):
        for k in ("answer", "answers"):
            if k in item["meta"]:
                v = item["meta"][k]
                return v if isinstance(v, list) else [v]

    # fallback: empty list (no reference found)
    return []


Testing:   0%|          | 0/930 [00:00<?, ?it/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may 

KeyboardInterrupt: 

In [ ]:
###############

In [ ]:
import torch, csv, time
from tqdm.auto import tqdm

# ------- Config (tune these) -------
PREFERRED_BATCH = 4    # try this, fallback to 1 on OOM
TEST_BATCH_SIZE = PREFERRED_BATCH
GEN_ARGS_FAST = dict(max_new_tokens=32,   # try 16 or 32
                     num_beams=1,        # greedy (fast)
                     do_sample=False,
                     early_stopping=True,
                     use_cache=True)
SAVE_EVERY = 100       # save partial CSV every N examples
OUT_CSV = "test_predictions_fast.csv"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# -----------------------------------

# Helper to save incremental results
def save_partial(eids, preds, refs, path=OUT_CSV):
    header = True
    import os
    if os.path.exists(path):
        header = False
    with open(path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if header:
            writer.writerow(["example_id","prediction","references"])
        for eid, p, r in zip(eids, preds, refs):
            writer.writerow([eid, p, " ||| ".join(r)])

# Try to set a larger batch size, fallback on OOM
try:
    test_loader = DataLoader(test_tok, batch_size=TEST_BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
except Exception as e:
    print("Could not create loader with batch", TEST_BATCH_SIZE, "-> fallback to 1. Err:", e)
    TEST_BATCH_SIZE = 1
    test_loader = DataLoader(test_tok, batch_size=1, shuffle=False, collate_fn=collate_fn)


In [ ]:

# If you still OOM during generation, fallback inside loop
model.eval()
all_preds = []
all_refs = []
example_ids = []
count = 0
start_time = time.time()

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing (fast)"):
        # move tensors to device
        batch_device = {}
        for k,v in batch.items():
            if isinstance(v, torch.Tensor):
                batch_device[k] = v.to(DEVICE)
            else:
                batch_device[k] = v

        # If your collate_fn yields token tensors already padded, pass them directly:
        in_ids = batch_device.get("input_ids")
        attn = batch_device.get("attention_mask", None)

        # generate with try/except to fallback to smaller batch or greedy
        try:
            gen_out = model.generate(input_ids=in_ids, attention_mask=attn, **GEN_ARGS_FAST)
        except RuntimeError as e:
            # OOM or similar: fallback to single-sample generation
            torch.cuda.empty_cache()
            gen_out_list = []
            for i in range(in_ids.shape[0]):
                single_in = { "input_ids": in_ids[i:i+1], "attention_mask": attn[i:i+1] if attn is not None else None }
                out = model.generate(**{k:v for k,v in single_in.items() if v is not None}, **GEN_ARGS_FAST)
                gen_out_list.append(out[0])
            gen_out = torch.stack(gen_out_list, dim=0)

        # decode preds and extract refs (same extraction logic as before)
        for i in range(gen_out.shape[0]):
            gen_ids = gen_out[i]
            pred_text = tokenizer.decode(gen_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            all_preds.append(pred_text)

            # extract references using same multi-key strategy
            ref_texts = []
            if "raw" in batch and isinstance(batch["raw"], list):
                refs = extract_references_from_item(batch["raw"][i])
                if refs:
                    ref_texts = refs
            if not ref_texts:
                for key in ("answers","answer_text","labels_text","gold_answer"):
                    if key in batch and isinstance(batch[key], list):
                        try:
                            val = batch[key][i]
                            if isinstance(val, list):
                                ref_texts = val
                            elif isinstance(val, str):
                                ref_texts = [val]
                            break
                        except Exception:
                            pass
            if not ref_texts and "labels" in batch and isinstance(batch["labels"], torch.Tensor):
                try:
                    lab = batch["labels"][i]
                    txt = tokenizer.decode(lab, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                    ref_texts = [txt]
                except Exception:
                    pass
            if not ref_texts:
                ref_texts = [""]

            all_refs.append(ref_texts)
            if "id" in batch and isinstance(batch["id"], (list,tuple)):
                example_ids.append(batch["id"][i])
            else:
                example_ids.append(len(example_ids))

            count += 1
            # periodic save to avoid losing progress
            if count % SAVE_EVERY == 0:
                save_partial(example_ids[-SAVE_EVERY:], all_preds[-SAVE_EVERY:], all_refs[-SAVE_EVERY:], OUT_CSV)

# final save
save_partial(example_ids, all_preds, all_refs, OUT_CSV)
elapsed = time.time()-start_time
print(f"Done. Generated {len(all_preds)} examples in {elapsed/60:.1f} min. Partial results saved to {OUT_CSV}")


Testing (fast):   0%|          | 0/233 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are

Done. Generated 930 examples in 45.5 min. Partial results saved to test_predictions_fast.csv


In [ ]:
# ---------- after the generation loop ----------
total_em = 0
total_f1 = 0.0
n = len(all_preds)

for pred, refs in zip(all_preds, all_refs):
    # Multi-reference case: pick the best match
    em_for_example = max(exact_match_score(pred, r) for r in refs)
    f1_for_example = max(f1_score(pred, r) for r in refs)

    total_em += 1 if em_for_example else 0
    total_f1 += f1_for_example

em = 100.0 * total_em / max(1, n)
avg_f1 = 100.0 * total_f1 / max(1, n)

print(f"Test results -> EM: {em:.2f} | F1: {avg_f1:.2f} (n={n})")


Test results -> EM: 0.00 | F1: 0.00 (n=930)


In [ ]:
len(all_preds)

NameError: name 'all_preds' is not defined

In [ ]:
## DEUG::


In [ ]:
# Mini test loop for debugging (only first N batches)
N_BATCHES = 2   # change to 3 or 5 if you want more
mini_preds, mini_refs, mini_ids = [], [], []

model.eval()
with torch.no_grad():
    for b_idx, batch in enumerate(test_loader):
        if b_idx >= N_BATCHES:
            break

        batch_device = {k: (v.to(DEVICE) if isinstance(v, torch.Tensor) else v)
                        for k,v in batch.items()}
        in_ids = batch_device.get("input_ids")
        attn = batch_device.get("attention_mask", None)

        # generation
        try:
            gen_out = model.generate(input_ids=in_ids, attention_mask=attn, **GEN_ARGS)
        except RuntimeError:
            torch.cuda.empty_cache()
            gen_list = []
            for i in range(in_ids.size(0)):
                out = model.generate(input_ids=in_ids[i:i+1],
                                     attention_mask=attn[i:i+1] if attn is not None else None,
                                     **GEN_ARGS)
                gen_list.append(out[0])
            gen_out = torch.stack(gen_list, dim=0)

        # decode + references
        for i in range(gen_out.size(0)):
            pred_text = tokenizer.decode(gen_out[i],
                                         skip_special_tokens=True,
                                         clean_up_tokenization_spaces=True)
            mini_preds.append(pred_text)

            # references (reuse your helper)
            ref_texts = extract_references_from_item(batch["raw"][i]) if "raw" in batch else []
            if not ref_texts and "labels" in batch:
                lab = batch["labels"][i]
                try:
                    txt = tokenizer.decode(lab, skip_special_tokens=True,
                                           clean_up_tokenization_spaces=True)
                    ref_texts = [txt]
                except:
                    ref_texts = [""]
            if not ref_texts:
                ref_texts = [""]

            mini_refs.append(ref_texts)
            mini_ids.append(b_idx*len(in_ids)+i)

# show a couple of predictions & gold answers
for i in range(min(5, len(mini_preds))):
    print(f"\n--- EX {mini_ids[i]} ---")
    print("RAW PRED:", repr(mini_preds[i][:300]))
    print("REFS:", mini_refs[i])


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


KeyboardInterrupt: 

In [ ]:
# Peek into one raw item from dataset
for batch in test_loader:
    print(batch.keys())
    print(batch)
    break


dict_keys(['input_ids', 'attention_mask', 'labels'])
{'input_ids': tensor([[ 13628,  47686,  59756,  ..., 128001, 128001, 128001],
        [ 13628,  47686,  59756,  ..., 128001, 128001, 128001],
        [ 13628,  47686,  59756,  ..., 128001, 128001, 128001],
        [ 13628,  47686,  59756,  ...,  44735, 115625, 128001]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[  -100,   -100,   -100,  ...,   -100,   -100,   -100],
        [  -100,   -100,   -100,  ...,   -100,   -100,   -100],
        [  -100,   -100,   -100,  ...,   -100,   -100,   -100],
        [  -100,   -100,   -100,  ...,  44735, 115625, 128001]])}


In [ ]:
tokenizer.decode(batch['input_ids'][0])

'دستور: فقط بر اساس متن موجود، پاسخ کوتاه بده.\n\nمتن:\nباشگاه فوتبال رئال مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال موفق\u200cترین تیم تاریخ فوتبال اسپانیا و موفق\u200cترین تیم تاریخ فوتبال اروپا و موفق\u200cترین تیم فوتبال سدهٔ ۲۰ میلادی به انتخاب فیفا است. آن\u200cها دارای رکورد ۳۴ بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، ۱۳ قهرمانی در لیگ قهرمانان اروپا، ۲ جام یوفا، ۴ سوپرکاپ اروپا و ۴ قهرمانی در جام باشگاه\u200cهای جهان\n\nسوال: پایتخت اسپانیا کجاست؟\n\nپاسخ:مادرید<|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of

In [ ]:
GEN_ARGS_FAST = {
    "max_new_tokens": 64,
    "do_sample": False,
    "num_beams": 1,
    "temperature": 1.0,
    "top_p": 1.0,
    "pad_token_id": tokenizer.pad_token_id,
    "eos_token_id": tokenizer.eos_token_id,
}


In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# assume test_loader, tokenizer, model, GEN_ARGS_FAST defined
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def prompt_from_encoded(input_ids, labels, tokenizer):
    """
    Given one sample's input_ids (tensor) and labels (tensor),
    return the prompt string (decoded) that excludes the gold answer.
    """
    # find first position where label != -100 (start of answer)
    non100 = (labels != -100).nonzero(as_tuple=True)[0]
    if len(non100) > 0:
        start = non100[0].item()
        prefix_ids = input_ids[:start]
    else:
        # no label positions: use entire input (shouldn't happen for your test set)
        prefix_ids = input_ids

    # decode prefix (skip special tokens)
    prompt_text = tokenizer.decode(prefix_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return prompt_text

def extract_short_answer_from_generated(gen_text):
    # try to extract after 'پاسخ:' marker or last non-empty line
    if "پاسخ:" in gen_text:
        return gen_text.split("پاسخ:", 1)[1].strip()
    if "Answer:" in gen_text:
        return gen_text.split("Answer:", 1)[1].strip()
    # fallback: last non-empty line
    lines = [ln.strip() for ln in gen_text.splitlines() if ln.strip()]
    return lines[-1] if lines else gen_text.strip()

# mini evaluation loop (do small batch to debug)
N_BATCHES = 3      # check first few batches
all_preds = []
all_refs = []
ids = []

model.eval()
with torch.no_grad():
    for bidx, batch in enumerate(test_loader):
        if bidx >= N_BATCHES:
            break

        # batch contains tensors: input_ids, attention_mask, labels
        b_input_ids = batch["input_ids"]           # shape (B, L)
        b_labels = batch["labels"]                 # shape (B, L)

        # build prompt_texts (list of strings)
        prompt_texts = []
        for i in range(b_input_ids.size(0)):
            p = prompt_from_encoded(b_input_ids[i], b_labels[i], tokenizer)
            prompt_texts.append(p)

        # tokenize prompts into a padded batch
        enc = tokenizer(prompt_texts, return_tensors="pt", padding=True, truncation=True)
        enc = {k: v.to(DEVICE) for k, v in enc.items()}

        # generate answers
        gen_out = model.generate(input_ids=enc["input_ids"], attention_mask=enc.get("attention_mask"), **GEN_ARGS_FAST)

        # decode and clean
        for i in range(gen_out.size(0)):
            gen_text_full = tokenizer.decode(gen_out[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
            short_answer = extract_short_answer_from_generated(gen_text_full)
            all_preds.append(short_answer)

            # extract reference(s) from original batch (use labels to decode gold)
            # decode gold answer tokens from labels (where labels != -100)
            lab = b_labels[i]
            lbl_positions = (lab != -100).nonzero(as_tuple=True)[0]
            if len(lbl_positions) > 0:
                start = lbl_positions[0].item()
                # gather all label token ids from start until first -100 or end
                gold_ids = lab[start:]
                # remove trailing -100 (if any) and decode
                gold_ids = gold_ids[gold_ids != -100]
                gold_text = tokenizer.decode(gold_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                all_refs.append([gold_text])
            else:
                all_refs.append([""])

            ids.append(bidx * b_input_ids.size(0) + i)

# Print small sample
for i in range(min(10, len(all_preds))):
    print("ID", ids[i])
    print("PRED:", all_preds[i])
    print("REFS:", all_refs[i])
    print("---")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


ID 0
PRED: باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال مادرید در سال ۱۹۰۰ تأسیس شده و در حال حاضر در لیگ دسته اول فوتبال اسپانیا بازی می‌کند.
REFS: ['مادرید']
---
ID 1
PRED: باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال موفق‌ترین تیم تاریخ فوتبال اسپانیا و موفق‌ترین تیم تاریخ فوتبال اروپا و موفق‌ترین تیم فوتبال سدهٔ ۲۰ میلادی به انتخاب فیفا است. آن‌ها دارای رکورد ۳۴ بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، ۱۳ قهرمانی در لیگ قهرمانان اروپا، ۲ جام یوفا، ۴ سوپرکاپ اروپا و ۴ قهرمانی در جام باشگاه‌های جهان هستند. رئال مادرید رکورددار قهرمانی در لیگ قهرمانان اروپا با ۱۳ قهرمانی است.
REFS: ['فیفا']
---
ID 2
PRED: سوال: رئال مادرید چند بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا،
REFS: ['۱۳']
---
ID 3
PRED: به معنی «سلطنتی»
REFS: ['سلطنتی']
---
ID 4
PRED: دستور: فقط بر اساس متن موجود، پاسخ کوتاه در متن موجود نیست

In [ ]:
# reuse normalize_persian, exact_match_score, f1_score from earlier messages
total_em = 0
total_f1 = 0.0
n = len(all_preds)

for pred, refs in zip(all_preds, all_refs):
    em_for_example = max(exact_match_score(pred, r) for r in refs)
    f1_for_example = max(f1_score(pred, r) for r in refs)
    total_em += 1 if em_for_example else 0
    total_f1 += f1_for_example

print(f"EM: {100*total_em/max(1,n):.2f}, F1: {100*total_f1/max(1,n):.2f}, n={n}")


EM: 16.67, F1: 22.49, n=12


## Another try

In [ ]:
import torch, time, csv
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
import collections, re, string, os

# ----------------- Config (tune if needed) -----------------
TEST_BATCH_SIZE = 4           # depends on your GPU memory; set lower if OOM
GEN_ARGS_FAST = dict(
    max_new_tokens=32,
    num_beams=1,
    do_sample=False,
    use_cache=True,
    pad_token_id=getattr(tokenizer, "pad_token_id", None) or tokenizer.eos_token_id,
    eos_token_id=getattr(tokenizer, "eos_token_id", None),
)
SAVE_EVERY = 200              # save to CSV every N examples
OUT_CSV = "test_predictions_final.csv"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# ----------------------------------------------------------

# ensure left padding for decoder-only models
tokenizer.padding_side = "left"
if getattr(tokenizer, "pad_token_id", None) is None:
    tokenizer.pad_token = tokenizer.eos_token

# small helpers (Persian-aware normalization)
def normalize_persian(s):
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = s.replace("ي", "ی").replace("ى", "ی").replace("ك", "ک")
    s = s.replace("ؤ", "و").replace("إ", "ا").replace("أ", "ا").replace("ٱ", "ا")
    s = re.sub(r'[\u064B-\u0652\u0670\u06D6-\u06ED]', '', s)
    pers_punc = "؟،؛«»”“—…-ـ·"
    all_punc = set(string.punctuation + pers_punc)
    s = "".join(ch for ch in s if ch not in all_punc)
    s = " ".join(s.split())
    return s

def exact_match_score(pred, truth):
    return normalize_persian(pred) == normalize_persian(truth)

def f1_score(pred, truth):
    p_tokens = normalize_persian(pred).split()
    t_tokens = normalize_persian(truth).split()
    if len(p_tokens) == 0 and len(t_tokens) == 0:
        return 1.0
    if len(p_tokens) == 0 or len(t_tokens) == 0:
        return 0.0
    common = collections.Counter(p_tokens) & collections.Counter(t_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0
    prec = num_same / len(p_tokens)
    rec = num_same / len(t_tokens)
    return 2 * prec * rec / (prec + rec)

# helpers for building prompt and extracting short answer
def prompt_from_encoded(input_ids, labels):
    non100 = (labels != -100).nonzero(as_tuple=True)[0]
    if len(non100) > 0:
        start = non100[0].item()
        prefix_ids = input_ids[:start]
    else:
        prefix_ids = input_ids
    # decode prefix (keep special tokens? skip for readability)
    return tokenizer.decode(prefix_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

def extract_short_answer_from_generated(gen_text):
    if "پاسخ:" in gen_text:
        return gen_text.split("پاسخ:", 1)[1].strip()
    if "Answer:" in gen_text:
        return gen_text.split("Answer:", 1)[1].strip()
    lines = [ln.strip() for ln in gen_text.splitlines() if ln.strip()]
    return lines[-1] if lines else gen_text.strip()

# incremental saver (appends)
def save_partial(eids, preds, refs, path=OUT_CSV):
    header = not os.path.exists(path)
    with open(path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if header:
            writer.writerow(["example_id", "prediction", "references"])
        for eid, p, r in zip(eids, preds, refs):
            writer.writerow([eid, p, " ||| ".join(r)])


In [ ]:

# ------------------ Evaluation loop (full test) ------------------
test_loader = DataLoader(test_tok, batch_size=TEST_BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

model.eval()
total_em = 0
total_f1 = 0.0
count = 0
all_preds_for_save = []
all_refs_for_save = []
all_ids_for_save = []

start_time = time.time()
with torch.no_grad():
    for batch_idx, batch in enumerate(tqdm(test_loader, desc="Testing")):
        # we have batch["input_ids"], batch["labels"]
        b_input_ids = batch["input_ids"]            # (B, L)
        b_labels = batch["labels"]                  # (B, L)

        # build prompts
        prompt_texts = [prompt_from_encoded(b_input_ids[i], b_labels[i]) for i in range(b_input_ids.size(0))]

        # tokenize prompts with left padding
        enc = tokenizer(prompt_texts, return_tensors="pt", padding=True, truncation=True)
        enc = {k: v.to(DEVICE) for k, v in enc.items()}

        # generate (with OOM fallback)
        try:
            gen_out = model.generate(input_ids=enc["input_ids"], attention_mask=enc.get("attention_mask"), **GEN_ARGS_FAST)
        except RuntimeError as e:
            # fallback to single-sample generation for this batch
            torch.cuda.empty_cache()
            gen_list = []
            for i in range(enc["input_ids"].size(0)):
                single_in = {"input_ids": enc["input_ids"][i:i+1]}
                if enc.get("attention_mask") is not None:
                    single_in["attention_mask"] = enc["attention_mask"][i:i+1]
                out = model.generate(**single_in, **GEN_ARGS_FAST)
                gen_list.append(out[0])
            gen_out = torch.stack(gen_list, dim=0)

        # process generated outputs and compute metrics
        for i in range(gen_out.size(0)):
            gen_text_full = tokenizer.decode(gen_out[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
            pred_short = extract_short_answer_from_generated(gen_text_full)

            # decode gold answer from labels
            lab = b_labels[i]
            lbl_pos = (lab != -100).nonzero(as_tuple=True)[0]
            if len(lbl_pos) > 0:
                start = lbl_pos[0].item()
                gold_ids = lab[start:]
                gold_ids = gold_ids[gold_ids != -100]
                gold_text = tokenizer.decode(gold_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            else:
                gold_text = ""

            # compute metrics (multi-ref not expected here, so single reference used)
            em = exact_match_score(pred_short, gold_text)
            f1 = f1_score(pred_short, gold_text)
            total_em += 1 if em else 0
            total_f1 += f1

            # collect for saving
            example_id = batch_idx * b_input_ids.size(0) + i
            all_ids_for_save.append(example_id)
            all_preds_for_save.append(pred_short)
            all_refs_for_save.append([gold_text])

            count += 1
            # periodic save
            if count % SAVE_EVERY == 0:
                save_partial(all_ids_for_save[-SAVE_EVERY:], all_preds_for_save[-SAVE_EVERY:], all_refs_for_save[-SAVE_EVERY:], OUT_CSV)

# final save and summary
save_partial(all_ids_for_save, all_preds_for_save, all_refs_for_save, OUT_CSV)

elapsed = time.time() - start_time
n = max(1, count)
print(f"Done. Examples: {n} | Time: {elapsed/60:.2f} min")
print(f"EM: {100.0 * total_em / n:.2f} | F1: {100.0 * total_f1 / n:.2f}")

# optional: print a few examples
for i in range(min(10, len(all_preds_for_save))):
    print(f"ID {all_ids_for_save[i]} | PRED: {all_preds_for_save[i]} | REF: {all_refs_for_save[i]}")


Testing:   0%|          | 0/233 [00:00<?, ?it/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be

Done. Examples: 930 | Time: 82.29 min
EM: 27.63 | F1: 39.87
ID 0 | PRED: باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال مادرید در سال ۱۹۰۰ تأسیس شده و در حال حاضر در لیگ دسته اول فوتبال اسپانیا بازی می‌کند. | REF: ['مادرید']
ID 1 | PRED: باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال موفق‌ترین تیم تاریخ فوتبال اسپانیا و موفق‌ترین تیم تاریخ فوتبال اروپا و موفق‌ترین تیم فوتبال سدهٔ ۲۰ میلادی به انتخاب فیفا است. آن‌ها دارای رکورد ۳۴ بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، ۱۳ قهرمانی در لیگ قهرمانان اروپا، ۲ جام یوفا، ۴ سوپرکاپ اروپا و ۴ قهرمانی در جام باشگاه‌های جهان هستند. رئال مادرید رکورددار قهرمانی در لیگ قهرمانان اروپا با ۱۳ قهرمانی است. | REF: ['فیفا']
ID 2 | PRED: سوال: رئال مادرید چند بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، | REF: ['۱۳']
ID 3 | PRED: به معنی «سلطنتی» | REF: ['سلطنتی']
ID 4 | PRED: 